<a href="https://colab.research.google.com/github/kaveku/WEEK1-lab1/blob/main/Copy_of_logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this second part of the lab, we will implement a language identifier trained on the same data, but using Logistic Regression instead of Naive Bayes.

In [39]:
import io, sys, math
import numpy as np
from collections import defaultdict

This function is used to build the dictionary, or vocabulary, which is a mapping from strings (or words) to integers (or indices). This will allow to build vector representations of documents. 

In [40]:
def build_dict(filename, threshold=1):
    fin = io.open(filename, 'r', encoding='utf-8')
    word_dict, label_dict = {}, {}
    counts = defaultdict(lambda: 0)
    for line in fin:
        tokens = line.split()
        label = tokens[0]

        if not label in label_dict:
            label_dict[label] = len(label_dict)

        for w in tokens[1:]:
            counts[w] += 1
            
    for k, v in counts.items():
        if v > threshold:
            word_dict[k] = len(word_dict)
    return word_dict, label_dict

This function is used to load the training dataset, and build vector representations of the training examples. In particular, a document or sentence is represented as a bag of words. Each example correspond to a sparse vector ` x` of dimension `V`, where `V` is the size of the vocabulary. The element `j` of the vector `x` is the number of times the word `j` appears in the document.

In [41]:
def load_data(filename, word_dict, label_dict):
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    dim = len(word_dict)
    for line in fin:
        tokens = line.split()
        label = tokens[0]

        yi = label_dict[label]
        xi = np.zeros(dim)
        for word in tokens[1:]:
            if word in word_dict:
                wid = word_dict[word]
                xi[wid] += 1.0
        data.append((yi, xi))
    return data

First, let's implement the softmax function. Don't forget numerical stability!

In [42]:
def softmax(x):
    z = np.exp(x - x.max())/(np.sum(np.exp(x - x.max())))
    return z

In [43]:
logits = np.random.rand(1, 20) + 100
softmax(logits), np.sum(softmax(logits))

(array([[0.06030108, 0.03503717, 0.05830398, 0.03763061, 0.05861041,
         0.0636796 , 0.06458685, 0.06878542, 0.02923711, 0.0317777 ,
         0.03736389, 0.04837189, 0.07332137, 0.04122715, 0.03455068,
         0.05541553, 0.03608323, 0.05725159, 0.04748366, 0.06098108]]), 1.0)

Now, let's implement the main training loop, by using stochastic gradient descent. The function will iterate over the examples of the training set. For each example, we will first compute the loss, before computing the gradient and performing the update.

In [44]:
def sgd(w, data, niter):
    nlabels, dim = w.shape
    lr = 0.1
    n = len(data)

    for iter in range(niter):
        for i in range(n):
            x = data[i][1].reshape(-1, 1)
            y_hat = data[i][0]
            y = np.zeros(nlabels)

            y[y_hat] = 1
            y = y.reshape(-1, 1)

            val_s = softmax(y*(w.dot(x)))
            tmp = ((1 - val_s) * y)@(x.T)
            w = w + lr * tmp
        
    return w

The next function will predict the most probable label corresponding to example `x`, given the trained classifier `w`.

In [45]:
def predict(w, x):
  return w@x

Finally, this function will compute the accuracy of a trained classifier `w` on a validation set.

In [46]:
def compute_accuracy(w, validation_data):
    result = 0
    for data in validation_data:
        pred = predict(w, data[1])
        res = np.argmax(softmax(pred))
        if res == data[0]:
            result += 1
    return result / len(validation_data)

In [47]:
print("")
print("** Logistic Regression **")
print("")

word_dict, label_dict = build_dict("train1.txt")
train_data = load_data("train1.txt", word_dict, label_dict)
validation_data = load_data("valid1.txt", word_dict, label_dict)

nlabels = len(label_dict)
dim = len(word_dict)
w = np.zeros([nlabels, dim])
w = sgd(w, train_data, 5)
print("")
print("Validation accuracy: %.3f" % compute_accuracy(w, validation_data))
print("")


** Logistic Regression **


Validation accuracy: 0.908

